In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import math
import random
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# read demo point cloud provided by Open3D
pcd_point_cloud = o3d.data.PCDPointCloud()
pcd = o3d.io.read_point_cloud(pcd_point_cloud.path)
# function to visualize the point cloud
o3d.visualization.draw_geometries([pcd], zoom=1,
                                front=[0.4257, -0.2125, -0.8795],
                                lookat=[2.6172, 2.0475, 1.532],
                                up=[-0.0694, -0.9768, 0.2024])

# Finding the plane from RANSAC API

In [3]:
pcd_point_cloud = o3d.data.PCDPointCloud()
pcd = o3d.io.read_point_cloud(pcd_point_cloud.path)

plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud],
                                  zoom=0.8,
                                  front=[-0.4999, -0.1659, -0.8499],
                                  lookat=[2.1813, 2.0619, 2.0999],
                                  up=[0.1204, -0.9852, 0.1215])

Plane equation: -0.06x + -0.10y + 0.99z + -1.06 = 0


# Finding the plane using RANSAC Algortihm(from scratch)

In [4]:
class RansacModel:
    
    def __init__(self, point_cloud_data = None, curve_fitting_model = None, num_iter = 1000, threshold = 0.01, n=1, d=1):
        self.pcd = point_cloud_data                    # Point Cloud Data
        self.model = curve_fitting_model               # Model to fit the data
        self.k = num_iter                              # Maximum iterations allowed
        self.t = threshold                             # Threshold values to determine data points fit well to the model
        self.n = n                                     # Minimum number of data points to estimate model parameters
#         self.d = d                           # Number of close data points required to assert that a model fits well to the data
        
    def collinearity_check(self, AB, AC):
        # Neglecting 0.5 to avoid floating point computation
        area = np.linalg.norm(np.cross(AB,AC))
        if area == 0:
            return True # The points are collinear
        else:
            return False
    
    # Function to find equation of plane.
    def equation_plane(self, x1, y1, z1, x2, y2, z2, x3, y3, z3):
     
        a1 = x2 - x1
        b1 = y2 - y1
        c1 = z2 - z1
        a2 = x3 - x1
        b2 = y3 - y1
        c2 = z3 - z1
        a = b1 * c2 - b2 * c1
        b = a2 * c1 - a1 * c2
        c = a1 * b2 - b1 * a2
        d = (- a * x1 - b * y1 - c * z1)
        
        return [a,b,c,d]
    
           
    def fit(self):
        iterations = 0
        data_points = np.asarray(pcd.points)
        total_points = len(data_points)
        
        # We need to the best fit plane to our point cloud.
        # The equation of a plane is given by ax + by + cz + d = 0
        # So we need minimum 3 points to find the parameters a, b, c
        
        a_final = 0
        b_final = 0
        c_final = 0
        d_final = 0
        
        data_point_satisfy = -1
        
        selected_indices = []
        while iterations < self.k:

            data_point_satify_temp = 0
            inliers = []
            inliers_results = []
            
            
            
            area = True
            # Selecting n random points from the point cloud
            while area != False:

                random_indices = random.sample(range(1,total_points), self.n)
                # Check whether the three selected points are collinear or not
                AB = data_points[random_indices[1]] - data_points[random_indices[0]]
                AC = data_points[random_indices[2]] - data_points[random_indices[0]]
                area = self.collinearity_check(AB,AC)
                if random_indices in selected_indices:
                    area = True
                
            # Now we are ensured that the three points are not collinear

            selected_indices.append(random_indices) 
            
            # Creating a plane from the three randomly chosen points ax + by + cz + d = 0
            [x1,y1,z1] = data_points[random_indices[0]]
            [x2,y2,z2] = data_points[random_indices[1]]
            [x3,y3,z3] = data_points[random_indices[2]]
            
            [a,b,c,d] = self.equation_plane(x1, y1, z1, x2, y2, z2, x3, y3, z3)
            
            
            parameters = np.array([[a],[b],[c]])
            parameters_mod = math.sqrt(a*a + b*b + c*c)
            points_plane = np.matmul(data_points,parameters) + d*np.ones([total_points,1])
            distance_matrix = abs(points_plane/parameters_mod)
            

            data_point_satisfy_temp = np.count_nonzero(distance_matrix < self.t)

        
            if data_point_satisfy < data_point_satisfy_temp:

                data_point_satisfy = data_point_satisfy_temp

                [a_final, b_final, c_final, d_final] = [a,b,c,d]
#                 print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")
               
            iterations = iterations + 1
        
        plane_model = [a_final, b_final, c_final, d_final]
        return plane_model

In [13]:
if __name__ == '__main__':
    '''
       In our case we need to find the best plane that fits the 3D Point cloud
    '''
    n = 3 # Minimum number of points required to estimate a plane is 3
    k = 5000
    threshold = 0.01
    
    model = RansacModel(point_cloud_data=pcd,n=3,num_iter=k,threshold=threshold)

    plane_model= model.fit()
    a = plane_model[0]
    b = plane_model[1]
    c = plane_model[2]
    d = plane_model[3]
    print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")
    
    
    
    
    
    
    

Plane equation: -0.04x + -0.08y + 0.77z + -0.81 = 0


In [14]:

print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud],
                                  zoom=0.8,
                                  front=[-0.4999, -0.1659, -0.8499],
                                  lookat=[2.1813, 2.0619, 2.0999],
                                  up=[0.1204, -0.9852, 0.1215])

Plane equation: -0.04x + -0.08y + 0.77z + -0.81 = 0


# References
1.) [Plane Segmentation using RANSAC Open3d API](http://www.open3d.org/docs/latest/tutorial/Basic/pointcloud.html)<br>
2.) [RANSAC Algorithm](https://www.youtube.com/watch?v=Cu1f6vpEilg)